# Urban Sounds Classification

A numeric identifier of the sound class:
* 0 = air_conditioner
* 1 = car_horn
* 2 = children_playing
* 3 = dog_bark
* 4 = drilling
* 5 = engine_idling
* 6 = gun_shot
* 7 = jackhammer
* 8 = siren
* 9 = street_music

In [47]:
import pandas as pd

df = pd.read_csv("./UrbanSound8K.csv")
df

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   slice_file_name  8732 non-null   object 
 1   fsID             8732 non-null   int64  
 2   start            8732 non-null   float64
 3   end              8732 non-null   float64
 4   salience         8732 non-null   int64  
 5   fold             8732 non-null   int64  
 6   classID          8732 non-null   int64  
 7   class            8732 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 545.9+ KB


Aşağıdaki kod bloğunda da görüldüğü gibi veri seti içerisinde herhangi bir eksik değer bulunmamaktadır.

In [ ]:
df.isnull().sum()

Veri dağılımı aşağıda da görüldüğü gibi eşit değildir. Bu veri seti ile eğiten bir modelin gun_shot sesini doğru tahmin etme olasılığı en düşüktür. Bundan dolayı veri ön işlenirken ve model oluşturulurken buna dikkat edilmelidir. Veri buna göre manipüle edilmelidir.

In [ ]:
df['class'].value_counts()

**Opencv** kütüphanesi, specktrogramları okumak, renk uzayını değiştirmek, yeniden boyutlandırmak ve normalize etmek için kullanılmıştır.

**OS** modülü, spektrogramların dosya yolunu bulmak, yeni dosyaya kaydetmek için kullanılmıştır.

**Random** modülü, görüntü-etiket şeklinde oluşturulan liste'nin içerisindeki verilerin sırasını random olacak şekilde karıştırarak bias ve varyans'ı minimize etmek için kullanılmıştır.

**Pandas** kütüphanesi, dataframe oluşturarak liste'lerden daha hızlı işlemler yapmak için kullanılmıştır.

**Numpy** kütüphanesi, dizi manipülasyon işlemleri için kullanılmıştır.

**Tensorflow** ve **Keras** kütüphaneleri, CNN modeli oluşturmak, label encoding ve model sonuçlarını gözlemlemek için kullanılmıştır.

In [ ]:
import cv2
import os
from os import makedirs
from os.path import join    
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from sklearn import metrics 

Spektrogramları yeniden boyutlandırmak, normalize etmek ve grayscale etmek için her bir işlem fonksiyonlara ayrılmıştır. Veri seti içerisinde görsellerin sol ve alt kısımlarında 3 piksellik beyaz bir çizgi bulunmaktadır. Model eğitimindeki başarıyı artırmak için bu çizgiler resize_img fonksiyonunda spektrogramlar yeniden boyutlandırılarak kaldırılmıştır.

In [ ]:
def normalize_img(img):
    result = cv2.normalize(img, None, 0, 225, norm_type=cv2.NORM_MINMAX)
    return result

def resize_img(img):
    dim = (128, 128)

    resized = resized[0:resized.shape[0] - 3, 3:resized.shape[1]] # clearing whiteness in the bottom and left frame of the image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) # resize image by scale percent
    return resized

Gri renk uzayına çevrilen görüntü, yeniden boyutlandırıldıktan sonra normalize edilip yeni klasöre kaydedilmektedir. Bu sırada yeni görüntü ve label değeri `data_list` isimli listeye kaydedilmektedir.

In [ ]:
def convert_image(source_path, dest_path, data_list):
    image = cv2.imread(source_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    resized_img = resize_img(image_gray)
    
    normalized_img = normalize_img(resized_img) # normalized the resized image

    class_id = dest_path.split("-")[1]
    data_list = data_list.append([normalized_img, class_id])

    cv2.imwrite(dest_path, normalized_img) # save normalized img to destination directory

Veri seti, `spectrograms` klösürü içerisinde label değerlerine göre klasörlenmiştir. Bu verileri sırası ile okunup gerekli on işleme adımları yapılarak `grayscaled_images` klasörüne kaydedilmiştir. Bu klasör çalışma dizininde bulunmuyorsa otomatik olarak oluşturulacaktır.

In [ ]:
pwd = os.getcwd()
source_dir = os.path.join(pwd, 'spectrograms')
dest_dir = os.path.join(pwd, 'grayscaled_images')
data_list = []

for dirpath, dirnames, filenames in os.walk(source_dir):
    for filename in filenames:
        if filename.endswith('.png'):
            source_path = os.path.join(dirpath, filename)
            dest_path = os.path.join(dest_dir, os.path.relpath(dirpath, source_dir), filename)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            convert_image(source_path, dest_path, data_list)

Veri ön işleme adımlarından sonra yeni görüntüler ve label değerleri `data_list` değişkeninde tutulmaktadır ve random kütüphanesin `shuffle` metodu sayesinde random bir şekilde karıştırılmıştır. Sonra da bu değerler bir dataframe'e eklenmiştir.

In [ ]:
random.shuffle(data_list)
data_df = pd.DataFrame(data_list, columns=['spectrogram', 'label'])
data_df.head()

Dataframe içerisindeki değerleri eğitime hazır hale getirmek için görüntü ve label olmak üzere iki farklı değişkende saklanmaktadır.

In [ ]:
df_ = pd.read_csv("./UrbanSound8K.csv")
df = df_.copy()

print(data_df.shape)
print(df.shape)

X = data_df["spectrogram"]
print(X.head())

y = data_df["label"].astype(int)
print(y.head())

X değişkeni içerisideki verilerin boyutu aşağıdaki kod bloğunun çıktısında görüldüğü gibi 128x128'dir.

In [ ]:
X[0].shape

X ve y değişkenlerini tek boyuta indirmek ve veri setini split etmeye hazır hale getirmek için aşağıdaki işlem yapılmıştır.

In [ ]:
X_data = X
y_data = y
index_list = []
for i in range(len(X_data)):
    if X_data[i].shape != (128, 128):
        X_data = X_data.drop([i])
        y_data = y_data.drop([i])
        index_list.append(i)
        
        
X_data = X_data.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)

print(f"X shape: {X_data.shape}")
print(f"y shape: {y_data.shape}")

Veri seti **%80 train**, **%10 test** ve **%10 validation** olacak şekilde split edilmiştir ve label değerleri kategorik değerlere dönüştürülmüştür.

In [ ]:
# TRAIN
X_train = X_data[int(len(X_data) * .00) : int(len(X_data) * .80)].to_list()
X_train = np.asarray(X_train)
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)

y_train = y_data[int(len(y_data) * .00) : int(len(y_data) * .80)].to_list()
y_train = np.asarray(y_train)
y_train = to_categorical(y_train)

# VALIDATION
X_val = X_data[int(len(X_data) * .80) : int(len(X_data) * .90)].to_list()
X_val = np.asarray(X_val)
X_val = X_val.reshape(X_val.shape[0], 128, 128, 1)

y_val = y_data[int(len(y_data) * .80) : int(len(y_data) * .90)].to_list()
y_val = np.asarray(y_val)
y_val = to_categorical(y_val)

# TEST
X_test = X_data[int(len(X_data) * .90) : int(len(X_data) * 100)].to_list()
X_test = np.asarray(X_test)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

y_test = y_data[int(len(y_data) * .90) : int(len(y_data) * 100)].to_list()
y_test = np.asarray(y_test)
y_test = to_categorical(y_test)

# NORMALIZATION
X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

In [ ]:
input_shape = (128,128,1)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding="same", activation="relu", input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="same", activation="relu", input_shape=input_shape))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="same", activation="relu", input_shape=input_shape))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(10, activation="softmax"))

model.summary()

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

results = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

In [ ]:
loss, acc = model.evaluate(X_test, y_test) 
print("Loss: ",loss)
print("Accuracy", acc)